In [ ]:
# Optional: setup NoTexBook theme
%load_ext notexbook
%texify

**Adapted from**: [Ch1](https://github.com/uvm-plaid/programming-dp/blob/master/notebooks/ch1.ipynb)

# De-Identification

### Dataset

The dataset is based on census data. The personally identifiable information (**PII**) is made up.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
DATASET_URL = "https://raw.githubusercontent.com/uvm-plaid/programming-dp/master/notebooks/adult_with_pii.csv"

In [ ]:
adult = pd.read_csv(DATASET_URL)
adult.head()

# De-identification

*De-identification* is the process of removing *identifying information* from a dataset. The term *de-identification* is sometimes used as a synonym for other terms like *anonymization* or *pseudonymization*.

> **Learning Objectives**
> - Define the following concepts:
>   - De-identification & Re-identification
>   - Identifying information / personally identifying information
> - Learn Example of (Data) Attacks
>   - Linking & Differencing Attacks
>   - Understand limitations of aggregate statistics

### Personal Identifiable Information

Identifying information has no formal definition. It is usually understood to be information which would be used to identify us uniquely in the course of daily life - name, address, phone number, e-mail address, etc. 

As we will see later, it's *impossible* to formalize the concept of identifying information, because *all* information is identifying. 

The term **personally identifiable information** (`PII`) is often used as a synonym for "identifying information".

**Q**: How should we **de-identify** information? 

**A**: Easy - we just remove the columns that contain identifying information!

In [ ]:
adult_de_identified = adult.copy().drop(columns=['Name', 'SSN'])
adult_de_identified.head(1)

> 💡 We'll save some of the identifying information for later, when we'll use it as *auxiliary data* to perform a *re-identification* attack.

Selected **PPI**s in the dataset:

In [ ]:
adult_pii = adult[['Name', 'SSN', 'DOB', 'Zip']]
adult_pii.head()

## Linking Attacks

Imagine we want to determine the income of a friend from our de-identified data. 

Names have been removed, but we happen to know some _auxiliary information_ about our friend. 

Our friend's name is **Karrie Trusslove**, and we know Karrie's date of birth and zip code.

To perform a simple **linking attack**, we look at the _overlapping columns between the dataset_ we're trying to attack, and the auxiliary data we know.

In this case, both datasets have dates of birth and zip codes.

We look for rows in the dataset we're attacking with `dates of birth` and `zip codes` that match Karrie's `date of birth` and `zip code`.

If there is **only one** such row, we've found Karrie's row in the dataset we're attacking. 

In databases, this is called a **JOIN** of two tables, and we can do it in Pandas using `merge`.

In [ ]:
karries_row = adult_pii[adult_pii['Name'] == 'Karrie Trusslove']
karries_row

In [ ]:
auxiliary_info = karries_row[["DOB", "Zip"]]
auxiliary_info.head()

In [ ]:

pd.merge(auxiliary_info, adult_de_identified, left_on=['DOB', 'Zip'], right_on=['DOB', 'Zip'])

> 💡 Indeed, there is only one row that matches.
> We have used **auxiliary data** to re-identify an individual in a de-identified dataset, and we're able to infer that Karrie's income is less than `$50k`.

### How Hard is it to Re-Identify Karrie?

This scenario is made up, but linking attacks are surprisingly easy to perform in practice.

How easy? It turns out that in many cases, just one data point is sufficient to pinpoint a row!

In [ ]:
pd.merge(auxiliary_info, adult_de_identified, left_on=['Zip'], right_on=['Zip'])

So ZIP code is sufficient **by itself** to allow us to re-identify Karrie. What about date of birth?

In [ ]:
pd.merge(auxiliary_info, adult_de_identified, left_on=['DOB'], right_on=['DOB'])

This time, there are three rows returned - and we don't know which one is the real Karrie. 

**But we've still learned a lot about our dataset!**

- We know that there's a 2/3 chance that Karrie's income is less than $50k
- We can look at the differences between the rows to determine what additional auxiliary information would *help* us to distinguish them (e.g. sex, occupation, marital status)

#### Is Karrie Special?

How hard is it to re-identify others in the dataset? 

Is Karrie especially easy or especially difficult to re-identify? 

A good way to understand the effectiveness of this type of attack is to look at how **selective** certain pieces of data are.

In other words, how good they are at narrowing down the set of potential rows which may belong to the target individual.

For example, is it common for `birth dates` to occur more than once ?

We'd like to get an idea of how many dates of birth are likely to be useful in performing an attack, which we can do by looking at how common "unique" dates of birth are in the dataset.

In [ ]:
adult_pii['DOB'].value_counts().hist()

plt.title("How selective is date of birth as PII ?")
plt.xlabel('Number of Dates of Birth')
plt.ylabel('Number of Occurrences');

The histogram above shows that *the vast majority* of dates of birth occur 1, 2, or 3 times in the dataset, and *no date of birth* occurs more than 8 times. 

This means that date of birth is **fairly selective** - it's effective in narrowing down the possible records for an individual.

#### Quick Exercise:

Let's try to repeat the experiment with `ZIP` codes:

In [ ]:
# your code here


<details>

<summary>Solution</summary>

```python
adult_pii['Zip'].value_counts().hist()

plt.title("How selective is date of birth as PII ?")
plt.xlabel('Number of ZIP Codes')
plt.ylabel('Number of Occurrences');
```
</details>

<details>

<summary>Considerations</summary>

The results when using ZIP code happens to be even worse: ZIP code is *very* selective in this dataset. 

Nearly all the ZIP codes occur only once.
</details>

### How Many People can we Re-Identify?

**Q**: In this dataset, how many people can we re-identify uniquely?

We can use our auxiliary information to find out!

First, let's see what happens with just `dates of birth`.

We want to know how many *possible identities* are returned for each data record in the dataset.

In [ ]:
attack = pd.merge(adult_pii, adult_de_identified, left_on=['DOB'], right_on=['DOB'])
attack['Name'].value_counts().hist();

plt.title("How many records can we identify with DoB ?")
plt.xlabel("Entries count")
plt.ylabel("Number of Records")
plt.show()


The above histogram shows the **number of records with each number of possible identities**.

The results show that we can uniquely identify almost `7,000` of the data records (out of about `32,000`), and an additional `10,000` data records are narrowed down to **only two** possible identities.

So it's not possible to re-identify a majority of individuals using *just* date of birth. 

What if we collect more information, to narrow things down further? 

If we use **both** `date of birth` and `ZIP`, we're able to do much better. 

In fact, we're able to uniquely re-identify basically the whole dataset.

In [ ]:
attack = pd.merge(adult_pii, adult_de_identified, left_on=['DOB', 'Zip'], right_on=['DOB', 'Zip'])

attack['Name'].value_counts().hist();
plt.title("How many records can we identify with DoB & ZIP ?")
plt.xlabel("Entries count")
plt.ylabel("Number of Records")
plt.show()

> 💡 When we use both pieces of information, we can re-identify **essentially everyone**. 

This is a surprising result, since we generally assume that many people share the same birthday, and many people live in the same ZIP code.

It turns out that the *combination* of these factors is **extremely** selective.

According to Latanya Sweeney's work[1](#fn1), 87% of people in the US can be uniquely re-identified by the combination of date of birth, gender, and ZIP code.

<span id="fn1">**[1]**: Sweeney, L, _Simple Demographics Often Identify People Uniquely_ [link](https://dataprivacylab.org/projects/identifiability/)</span>

Let's just check that we've actually re-identified *everyone*, by printing out the number of possible data records for each identity:

In [ ]:
attack['Name'].value_counts(ascending=False).head()

Looks like we missed two people! In other words, in this dataset, only **two people** share a combination of ZIP code and date of birth.

## Aggregation

Another way to prevent the release of private information is to release only **aggregate** date.

In [ ]:
# Let's take the Age as an example
adult['Age'].mean()

### Problem of Small Groups

In many cases, aggregate statistics are broken down into smaller groups. 

For example, we might want to know the average age of people with a particular education level.

In [ ]:
adult[['Education-Num', 'Age']].groupby('Education-Num').mean().head(3)

Aggregation is supposed to _improve privacy_ because it's hard to identify the contribution of a particular individual to the aggregate statistic. 

But what if we aggregate over a group with just *one person* in it? 

In that case, the aggregate statistic reveals one person's age *exactly*, and provides no privacy protection at all! 

In our dataset, most individuals have a unique `ZIP` code - so if we compute the average age by ZIP code, then most of the "averages" actually reveal an individual's exact age.

In [ ]:
adult[['Zip', 'Age']].groupby('Zip').mean().head()

The US Census Bureau, for example, releases aggregate statistics at the [*block level*](https://www.census.gov/newsroom/blogs/random-samplings/2011/07/what-are-census-blocks.html). 

Some census blocks have large populations, but some have a population of zero! 

The situation above, where small groups prevent aggregation from hiding information about individuals, turns out to be quite common.

How big a group is "big enough" for aggregate statistics to help? 

It's hard to say - it depends on the data and on the attack - so it's challenging to build confidence that aggregate statistics are really privacy-preserving. 

However, even very large groups do not make aggregation completely robust against attacks, as we will see next.

### Differencing Attacks

The problems with aggregation get even worse when you release multiple aggregate statistics over the same data. 

For example, consider the following two summation queries over large groups in our dataset (the first over the whole dataset, and the second over all records except one):

In [ ]:
adult['Age'].sum()

In [ ]:
adult[adult['Name'] != 'Karrie Trusslove']['Age'].sum()

If we know both answers, we can simply take the difference and determine Karrie's age completely! 

This kind of attack can proceed even if the aggregate statistics are over *very large groups*.

In [ ]:
adult['Age'].sum() - adult[adult['Name'] != 'Karrie Trusslove']['Age'].sum()

#### Take away message

(This is a recurring theme:)

1. Releasing *data* that is useful makes ensuring *privacy* very difficult.

2. Distinguishing between *malicious* and *non-malicious* queries is not possible (in general).

## Summary

- A *Linking attack* involves combining *auxiliary data* with *de-identified data* to *re-identify* individuals.

- In the simplest case, a linking attack can be performed via a *join* of two tables containing these datasets.

- Simple linking attacks are surprisingly effective:
  - Just a single data point is sufficient to narrow things down to a few records
  - The narrowed-down set of records helps suggest additional auxiliary data which might be helpful
  - Two data points are often good enough to re-identify a huge fraction of the population in a particular dataset
  - Three data points (gender, ZIP code, date of birth) uniquely identify 87% of people in the US



- Releasing aggregate statistics is another way to not disclose sensitive information.

  - But sometimes aggregating per groups could lead to sensitive data leakage nonetheless.
  - A *Differencing Attack* involves gathering some information out from auxiliary info and aggregate statistics. 